In [30]:
%pip install --upgrade --quiet  langchain langchain_experimental langchain-openai
!pip install openai
# Set env var OPENAI_API_KEY or load from a .env file:

import os
os.environ["OPENAI_API_KEY"] = ""

from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

In [31]:
class FirstApartmentPurchase(BaseModel):
    age: int
    monthly_income: float
    apartment_interest_rate: float
    down_payment: float
    apartment_price: float
    loan_term_years: int
    monthly_expenses: float
    credit_score: int
    employment_status: str

In [32]:
examples = [
    {
        "example": """Age: 30, Monthly Income: $4000, Apartment Interest Rate: 4.5%,
        Down Payment: $25000, Apartment Price: $200000, Loan Term (Years): 25,
        Monthly Expenses: $1500, Credit Score: 700, Employment Status: Employed"""
    },
    {
        "example": """Age: 25, Monthly Income: $3000, Apartment Interest Rate: 3.8%,
        Down Payment: $20000, Apartment Price: $180000, Loan Term (Years): 20,
        Monthly Expenses: $1200, Credit Score: 650, Employment Status: Self-Employed"""
    },
    {
        "example": """Age: 28, Monthly Income: $4500, Apartment Interest Rate: 4.0%,
        Down Payment: $30000, Apartment Price: $250000, Loan Term (Years): 30,
        Monthly Expenses: $1800, Credit Score: 720, Employment Status: Employed"""
    },
]


In [33]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)

In [35]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=FirstApartmentPurchase,
    llm=ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0.7
    ),
    prompt=prompt_template,
)

In [37]:
synthetic_results = synthetic_data_generator.generate(
    subject="FirstApartmentPurchase",
    extra="the name must be chosen at random. Make it something you wouldn't normally choose.",
    runs=1000,
)

In [ ]:
print(synthetic_results)

In [38]:
import pandas as pd

# Supondo que os resultados sintéticos estejam em um formato de lista de dicionários
# onde cada dicionário representa um registro médico faturado
# Por exemplo:
# synthetic_results = [
#     {"patient_name": "John Doe", "amount_billed": 100.0, ...},
#     {"patient_name": "Jane Smith", "amount_billed": 150.0, ...},
#     ...
# ]

# Convertendo os resultados sintéticos em um DataFrame do pandas
df = pd.DataFrame(synthetic_results)

# Especificando o caminho onde você deseja salvar o arquivo CSV
csv_file_path = "synthetic_FirstApartmentPurchase_records.csv"

# Salvando o DataFrame como um arquivo CSV
df.to_csv(csv_file_path, index=False)

print("Arquivo CSV salvo com sucesso:", csv_file_path)


Arquivo CSV salvo com sucesso: synthetic_FirstApartmentPurchase_records.csv


In [40]:
from google.colab import files

# Salvando o DataFrame como um arquivo CSV
df.to_csv(csv_file_path, index=False)

# Baixando o arquivo CSV
files.download(csv_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>